In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

torch.manual_seed(22)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_dir = '/Home/siv33/vbo084/DotaPicker/data/processed/'
model_dir = '/Home/siv33/vbo084/DotaPicker/saved_models/'

#fnames = ['dota_train_22.pt', 'dota_val_22.pt', 'dota_test_22.pt']
fnames = ['dota_train_23.pt', 'dota_val_23.pt', 'dota_test_23.pt']

In [ ]:
from utils import load_data
BATCH_SIZE = 128
train_dloader, val_dloader, test_dloader = load_data(data_dir, fnames, batch_size=BATCH_SIZE)

In [ ]:
#model = torch.load(os.path.join(model_dir, 'heropicker_final_23.pt'))

model = torch.load(os.path.join(model_dir, 'heropicker_final_23_nonsense.pt'))

In [ ]:
def eval(model, dloader, device):

    model.eval()
    pickban_hits = 0
    teamwin_hits = 0
    total_pickban_elements = 0
    total_teamwin_elements = 0

    with torch.no_grad():
        for batch in dloader:
            inputs = batch
            inputs = inputs.to(device)
            targets = inputs[:, 1:]
            scores = torch.argmax(model(inputs)[:, :-1, :], dim=-1)

            # Calculate picks and bans predictions
            pickban_scores, targets_pickbans = scores[:, :-1], targets[:, :-1]
            #print(f'Pickban scores: {pickban_scores[:25]}')
            #print(f'Target scores: {targets_pickbans[:25]}')
            pickban_hits += (pickban_scores == targets_pickbans).sum().item()
            total_pickban_elements += targets_pickbans.numel()

            # Calculates winrate scores
            teamwin_scores, targets_teamwin = scores[:, -1], targets[:, -1]
            teamwin_hits += (teamwin_scores == targets_teamwin).sum().item()
            total_teamwin_elements += targets_teamwin.numel()
            break

    pickban_acc = pickban_hits/total_pickban_elements
    win_acc = teamwin_hits/total_teamwin_elements

    print(f'Accuracy on picks and bans: {pickban_acc}')
    print(f'Accuracy on winning team: {win_acc}')

    return pickban_acc, win_acc

In [ ]:
a,b = eval(model, test_dloader, device)